In [1]:
from datasets import load_dataset, Dataset, DatasetDict 
from transformers import AutoTokenizer, DataCollatorWithPadding
import pandas as pd

dataset = load_dataset('sentiment140')
dictionary = {}
mapping_dict = {0:0, 2: 1, 4: 2}

test_dataset = dataset["test"].to_pandas()
train_negative_df = test_dataset.query("sentiment == 0")[:50]
train_neutral_df  = test_dataset.query("sentiment == 2")[:50]
train_positive_df = test_dataset.query("sentiment == 4")[:50]

dictionary["train"] = pd.concat([train_negative_df, train_neutral_df, train_positive_df])

dictionary["train"]["sentiment"] = dictionary["train"]["sentiment"].map(mapping_dict)

validation_dataset = dataset["test"].to_pandas()
validation_negative_df = validation_dataset.query("sentiment == 0")[50:100]
validation_neutral_df  = validation_dataset.query("sentiment == 2")[50:100]
validation_positive_df = validation_dataset.query("sentiment == 4")[50:100]

dictionary["validation"] = pd.concat([validation_negative_df, validation_neutral_df, validation_positive_df])

dictionary["validation"]["sentiment"] = dictionary["validation"]["sentiment"].map(mapping_dict)

train_dataset = Dataset.from_dict(dictionary["train"])
validation_dataset = Dataset.from_dict(dictionary["validation"])
dataset = DatasetDict({"train":train_dataset, "validation":validation_dataset})

/home/carlos/Documents/git/fine-tunning-tutorial/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.rename_column("sentiment", "label")

Map: 100%|██████████| 150/150 [00:00<00:00, 25132.65 examples/s]


In [3]:
from transformers import TrainingArguments

output_dir = "./output"
training_args = TrainingArguments(
  output_dir=output_dir,
  num_train_epochs=3,  # Number of training epochs
  per_device_train_batch_size=8,  # Batch size per GPU
  per_device_eval_batch_size=8,   # Batch size for evaluation per GPU
  warmup_steps=400,   # Number of warmup steps for learning rate scheduler
  weight_decay=0.01,   # Strength of weight decay
  logging_dir="./logs",   # Directory for storing logs
  logging_steps=100,   # Log every N steps
  evaluation_strategy="steps",   # Evaluation strategy during training
  eval_steps=200,   # Run evaluation every N steps
  save_total_limit=2,   # Only save the last N checkpoints
  save_steps=400,   # Save checkpoint every N steps
  load_best_model_at_end=True,   # Load the best model at the end of training
  metric_for_best_model="accuracy",   # Metric to use for the best model
)

print(training_args.output_dir)
print(training_args.num_train_epochs)

./output
3


In [41]:
from transformers import AutoModelForSequenceClassification, BertConfig, BertModel
import torch.nn as nn

class MyBertForSequenceClassification(nn.Module):
    def __init__(self, config):
        super(MyBertForSequenceClassification, self).__init__()
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.num_labels = config.num_labels

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            print(logits)
            print(logits.view(-1, self.num_labels))
            loss = loss_fct(logits, labels)
            #print(f"Logits view: {logits.view(-1, self.num_labels)}")
            #print(f"Labels view: {labels.view(-1)}")
            #print(f"Labels: {labels}")
            #print(f"Logits: {logits}")
            print(f"Loss: {loss}")
            return {"loss": loss} 
        else:
            return {"logits": logits}


config = BertConfig.from_pretrained(checkpoint)
config.num_labels = 3
model = MyBertForSequenceClassification(config)

In [42]:
from transformers import Trainer
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [43]:
trainer.train()

  4%|▎         | 2/57 [00:00<00:03, 14.04it/s]

8
tensor([[-0.4291,  0.0012, -0.4895],
        [-0.4646, -0.0839,  0.0284],
        [-0.1326, -0.2471, -0.1731],
        [-0.1289,  0.1501, -0.2593],
        [-0.2558,  0.2687, -0.2446],
        [-0.1555,  0.0538, -0.2914],
        [-0.3496,  0.0566, -0.1932],
        [-0.2588, -0.0428, -0.2448]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[-0.4291,  0.0012, -0.4895],
        [-0.4646, -0.0839,  0.0284],
        [-0.1326, -0.2471, -0.1731],
        [-0.1289,  0.1501, -0.2593],
        [-0.2558,  0.2687, -0.2446],
        [-0.1555,  0.0538, -0.2914],
        [-0.3496,  0.0566, -0.1932],
        [-0.2588, -0.0428, -0.2448]], device='cuda:0', grad_fn=<ViewBackward0>)
Loss: 1.1019532680511475
8
tensor([[-0.2998, -0.0330, -0.2948],
        [-0.5283, -0.0890, -0.3729],
        [-0.0803, -0.1065, -0.2383],
        [-0.1691, -0.4643, -0.1672],
        [-0.2517, -0.0080, -0.2059],
        [-0.7316, -0.0677, -0.0522],
        [-0.1975, -0.2162, -0.1926],
        [-0.3478,  0.0727,

  7%|▋         | 4/57 [00:00<00:04, 12.85it/s]

8
tensor([[-0.1272,  0.1044, -0.3857],
        [-0.1188,  0.0340, -0.0746],
        [-0.3326,  0.0920, -0.2519],
        [-0.3268,  0.0505, -0.0943],
        [-0.2655, -0.0747, -0.2208],
        [-0.1288,  0.0127, -0.3124],
        [-0.1745,  0.0300, -0.1289],
        [-0.3354, -0.0559, -0.0037]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[-0.1272,  0.1044, -0.3857],
        [-0.1188,  0.0340, -0.0746],
        [-0.3326,  0.0920, -0.2519],
        [-0.3268,  0.0505, -0.0943],
        [-0.2655, -0.0747, -0.2208],
        [-0.1288,  0.0127, -0.3124],
        [-0.1745,  0.0300, -0.1289],
        [-0.3354, -0.0559, -0.0037]], device='cuda:0', grad_fn=<ViewBackward0>)
Loss: 1.1360270977020264
8
tensor([[-0.2611, -0.0384, -0.0553],
        [-0.3478, -0.0095, -0.1853],
        [-0.3869, -0.0582, -0.1329],
        [-0.3819,  0.0711, -0.1593],
        [-0.4075,  0.1803, -0.2976],
        [-0.1606, -0.1210, -0.1194],
        [-0.6244, -0.1225, -0.6483],
        [-0.2849,  0.0435,

 14%|█▍        | 8/57 [00:00<00:03, 12.51it/s]

...

 98%|█████████▊| 56/57 [00:04<00:00, 12.47it/s]

8
tensor([[-0.4318, -0.2000, -0.0414],
        [-0.5001, -0.1103, -0.1526],
        [-0.3307, -0.1567, -0.0401],
        [-0.4163, -0.0111,  0.2162],
        [-0.3269, -0.1868, -0.1301],
        [-0.6385, -0.4182,  0.0234],
        [-0.3238, -0.0111,  0.0472],
        [-0.3288, -0.1761,  0.1160]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[-0.4318, -0.2000, -0.0414],
        [-0.5001, -0.1103, -0.1526],
        [-0.3307, -0.1567, -0.0401],
        [-0.4163, -0.0111,  0.2162],
        [-0.3269, -0.1868, -0.1301],
        [-0.6385, -0.4182,  0.0234],
        [-0.3238, -0.0111,  0.0472],
        [-0.3288, -0.1761,  0.1160]], device='cuda:0', grad_fn=<ViewBackward0>)
Loss: 1.0919084548950195
8
tensor([[-0.5317, -0.3676, -0.2356],
        [-0.4939, -0.2925,  0.2196],
        [-0.5475, -0.2722, -0.1556],
        [-0.6220, -0.5013,  0.1364],
        [-0.2905,  0.1567, -0.0502],
        [-0.2920, -0.3755,  0.1940],
        [-0.4147, -0.2794,  0.1395],
        [-0.4008, -0.2951,

                                               
100%|██████████| 57/57 [00:04<00:00, 12.58it/s]

{'train_runtime': 4.5312, 'train_samples_per_second': 99.311, 'train_steps_per_second': 12.579, 'train_loss': 1.1224767450700726, 'epoch': 3.0}


TrainOutput(global_step=57, training_loss=1.1224767450700726, metrics={'train_runtime': 4.5312, 'train_samples_per_second': 99.311, 'train_steps_per_second': 12.579, 'train_loss': 1.1224767450700726, 'epoch': 3.0})

In [46]:
predictions = trainer.predict(tokenized_datasets["validation"])
#print(predictions.predictions.shape, predictions.label_ids.shape)
predictions

45it [00:11,  3.01it/s]

8
tensor([[-0.2115, -0.4553,  0.0560],
        [-0.2710, -0.5812,  0.0960],
        [-0.2317, -0.4152,  0.0072],
        [-0.2819, -0.4031,  0.0526],
        [-0.2043, -0.3498,  0.0839],
        [-0.2934, -0.4640,  0.0550],
        [-0.2862, -0.4561,  0.0661],
        [-0.2165, -0.4650,  0.0917]], device='cuda:0')
tensor([[-0.2115, -0.4553,  0.0560],
        [-0.2710, -0.5812,  0.0960],
        [-0.2317, -0.4152,  0.0072],
        [-0.2819, -0.4031,  0.0526],
        [-0.2043, -0.3498,  0.0839],
        [-0.2934, -0.4640,  0.0550],
        [-0.2862, -0.4561,  0.0661],
        [-0.2165, -0.4650,  0.0917]], device='cuda:0')
Loss: 1.1596969366073608
8
tensor([[-0.2680, -0.4033,  0.0797],
        [-0.2604, -0.4825,  0.0770],
        [-0.2412, -0.3884,  0.0039],
        [-0.2991, -0.3717,  0.0626],
        [-0.2842, -0.5506,  0.0834],
        [-0.3248, -0.4259,  0.1072],
        [-0.3862, -0.3194,  0.1026],
        [-0.3880, -0.3094, -0.0078]], device='cuda:0')
tensor([[-0.2680, -0.4033,  0

57it [00:12,  5.91it/s]

8
tensor([[-0.3463, -0.4157,  0.0689],
        [-0.3520, -0.4393,  0.0252],
        [-0.2741, -0.4432,  0.0638],
        [-0.3301, -0.3863,  0.0469],
        [-0.3013, -0.4050,  0.0753],
        [-0.3685, -0.3282,  0.0345],
        [-0.3369, -0.3155,  0.0093],
        [-0.3720, -0.3308, -0.0025]], device='cuda:0')
tensor([[-0.3463, -0.4157,  0.0689],
        [-0.3520, -0.4393,  0.0252],
        [-0.2741, -0.4432,  0.0638],
        [-0.3301, -0.3863,  0.0469],
        [-0.3013, -0.4050,  0.0753],
        [-0.3685, -0.3282,  0.0345],
        [-0.3369, -0.3155,  0.0093],
        [-0.3720, -0.3308, -0.0025]], device='cuda:0')
Loss: 1.2747195959091187
8
tensor([[-0.2872, -0.4694,  0.0636],
        [-0.3378, -0.3716,  0.0590],
        [-0.2941, -0.4148,  0.1319],
        [-0.2994, -0.3850,  0.0459],
        [-0.3070, -0.4340,  0.0725],
        [-0.2707, -0.7005,  0.1009],
        [-0.3053, -0.4711,  0.0676],
        [-0.3120, -0.4848,  0.0427]], device='cuda:0')
tensor([[-0.2872, -0.4694,  0

ValueError: attempt to get argmax of an empty sequence

57it [00:28,  5.91it/s]